In [1]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: selenium in c:\users\suriija\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages (4.14.0)




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\Suriija\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\Suriija\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import pandas as pd

from datetime import datetime



In [3]:
# Set a user agent string to mimic a non-bot user
user_agent = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
)

# Configure Chrome options with the user agent
options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={user_agent}")

In [4]:
# Initialize the Chrome WebDriver with the configured options
options = Options()
# options.add_argument("--no-sandbox")
# options.add_argument('--headless=new')
# options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(
            latest_release_url='https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json',
            driver_version="124.0.6367.91").install()), options=options)

In [5]:
try: 
    #https://diga.bfarm.de/de/verzeichnis?type=%5B%22active%22%2C%22draft%22%2C%22retired%22%5D
    url = 'https://diga.bfarm.de/de/verzeichnis/'
    driver.get(url)
except: 
    print("Error wegen zeit über flutten ")

In [6]:
# Scroll through the page to load all elements
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(random.uniform(2, 4)) 

In [7]:
# Find all the elements on the page
elements = driver.find_elements("css selector", ".entity-app")

# Initialize an empty list to store the extracted data
data = []
table_data = []

batch_size = 10

# Set a counter to track the number of extracted records
counter = 0

In [8]:
elements = driver.find_elements("css selector", ".entity-app")
elements_count = len(elements)

# Try except geht nicht mehr in der ice ! 

for i in range(elements_count):
    time.sleep(2)
    print(f"Processing element {counter + 1}")
    info = {}
    
    # Fetch the elements again in case of stale reference
    elements = driver.find_elements("css selector", ".entity-app")
    #zweite except l'schen wenn man nicht im ice 
    try: 
        element = elements[i]
    except: 
        try:
            time.sleep(10)
            element = elements[i]
        except:
            for x in range(10):
                try: 
                    time.sleep(5)
                    element = elements[i]
                    break
                except: 
                    print("LOLO")
    # Click on the button to go to the detailed page
    button = element.find_element("css selector", ".entity-app__button")
    driver.execute_script("arguments[0].click();", button)
    try:
        WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".entity-app__app-type")))
    except:
        time.sleep(2)
      
    diga_id = {"diga_id": driver.current_url.split("/")[-1]} 
    combined_data = diga_id.copy()
    diga = diga_id.copy()
    # Re-fetch the elements on the home page
    elements = driver.find_elements("css selector", ".entity-app")

    # Wait for the detailed page to load
    info["app_name"]= driver.find_element("css selector", ".entity-app__header__name").get_attribute("innerText")
    info["app_owner"]= driver.find_element("css selector", ".entity-app__subheader").get_attribute("innerText")
    info["app_type"]= driver.find_element("css selector", ".entity-app__app-type").get_attribute("innerText")
    
    # Extract information from the detailed page
    # After extracting the main information from the detailed page, click the "Learn More" button to open the popup
    answers = {}
    button_elements = driver.find_elements("css selector", ".entity-app__info__more > button")
    for button_element in button_elements:
        driver.execute_script("arguments[0].click();", button_element)
        time.sleep(random.uniform(3, 4))  # Random delay to mimic human-like behavior

        # Wait for the popup to load
        # Initialize a dictionary to store the answers

        # Find all question and answer elements
        qa_elements = driver.find_elements("css selector",".entity-app-answers__questions__question, .entity-app-answers__questions__answer")

        # Loop through the question and answer elements
        for i in range(0, len(qa_elements), 2):
            question = qa_elements[i].text
            answer_element = qa_elements[i + 1]

            # Get the answer text from the answer element
            answer = answer_element.text
            print(answer, question)

            # Store the answer in the appropriate field in the answers dictionary based on the question
            if "Anwendungs-URL im Apple App Store" in question:
                answers["apple_app_url"] = answer
            elif "Versionsnummer iOS-App" in question:
                print('entered')
                answers["ios_version"] = answer
            elif "Hardwarekompatibilität iOS-App" in question:
                answers["ios_hardware_compatibility"] = answer
            elif "Softwarekompatibilität iOS-App" in question:
                answers["ios_software_compatibility"] = answer
            elif "Anwendungs-URL im Google Play Store" in question:
                answers["google_play_url"] = answer
            elif "Versionsnummer Android-App" in question:
                answers["android_version"] = answer
            elif "Hardwarekompatibilität Android-App" in question:
                answers["android_hardware_compatibility"] = answer
            elif "Softwarekompatibilität Android-App" in question:
                answers["android_software_compatibility"] = answer
            elif "Patientengruppe" in question:
                answers["patientengruppe"] = answer
            elif "Geeignete Altersgruppen" in question:
                answers["geeignete_altersgruppen"] = answer
            elif "Geeignete Geschlechter" in question:
                answers["geeignete_geschlechter"] = answer
            elif "Die DiGA ist nicht anzuwenden bei folgenden Diagnosen" in question:
                answers["diga_kontraindikationen"] = answer
            elif "Weitere nicht durch Kontraindikationen abgedeckte Ausschlusskriterien" in question:
                answers["weitere_ausschlusskriterien"] = answer
            elif "Herstellerpreis (Brutto)" in question:
                answers["herstellerpreis_brutto"] = answer
            elif "Vergütungs- / Höchstbetrag (Brutto)" in question:
                answers["verguetungs_hoechstbetrag_brutto"] = answer
            elif "Mehrkosten durch den Nutzenden der DiGA" in question:
                answers["mehrkosten_nutzenden"] = answer
            elif "Erläuterung Mehrkosten" in question:
                answers["erlaeuterung_mehrkosten"] = answer
            elif "Zusatzgeräte" in question:
                answers["zusatzgeraete"] = answer
            elif "Erläuterung der Zusatzgeräte" in question:
                answers["erlaeuterung_zusatzgeraete"] = answer
            elif "Kombinationsmöglichkeiten in Bezug auf Geräte" in question:
                answers["kombinationsmoeglichkeiten_geraete"] = answer
            elif "Diese DiGA ist verfügbar in den folgenden Sprachen" in question:
                answers["verfuegbar_sprachen"] = answer



        # Close the popup
        close_button = driver.find_element("css selector", ".modal-information__close")
        driver.execute_script("arguments[0].click();", close_button)

    href_list = ["Apple App Store","Google Play Store","Webanwendung"]
    href = [[".entity-app__info__list__items > div:nth-child(2) > a",".entity-app__info__list__items > div:nth-child(2)"],[".entity-app__info__list__items > div:nth-child(4) > a",".entity-app__info__list__items > div:nth-child(4)"]]

    des = ["app_summary",".entity-app__summary"]    
    answers[des[0]] = driver.find_element(By.CSS_SELECTOR, des[1]).text


    for y in href: 
        href_elements = driver.find_elements(By.CSS_SELECTOR, y[0])
        text_elements = driver.find_elements(By.CSS_SELECTOR, y[1])
        
        for href_element in zip(href_elements, text_elements):
            href  = href_element[0].get_attribute('href')
            text  = href_element[1].text 
            for x in href_list: 
                if x  == text: 
                    combined_data.update({x.lower().replace(" ","_"): href})
                    print (text)
                    print (href)
    
    fachkreise = str(driver.current_url)+"/fachkreise"
    driver.get(fachkreise)
    time.sleep(random.uniform(10, 13)) 

    qa_elements = driver.find_elements("css selector",".entity-app-answers__questions__question, .entity-app-answers__questions__answer")
    # Loop through the question and answer elements
    
    fachtkreis_ask = [['Plattform', 'platform'], ['Hardware', 'hardware'], ['Vorgesehene Mindestdauer der Anwendung der DiGA', 'min_application_duration'], ['Vorgesehene Höchstdauer der Anwendung der DiGA', 'max_application_duration'], ['Institutionskennzeichen (IK Nummer)', 'institution_code'], ['Datum des Beginns des Erprobungszeitraumes', 'start_trial_date'], ['Datum des Ende des Erprobungszeitraumes', 'end_trial_date'], ['Weitere Anmerkungen zum Preismodell', 'pricing_model_notes'], ['Erläuterung Mehrkosten', 'additional_costs_explanation'], ['Angaben zu Patientengruppen', 'patient_groups_information'], ['Weitere nicht durch Kontraindikationen abgedeckte Ausschlusskriterien', 'exclusion_criteria_not_covered'], ['Ärztinnen/ Ärzte', 'attending_physicians'], ['Erforderlichkeit vertragsärztlicher Tätigkeiten oder Tätigkeiten von Heilmittelerbringern oder Hebammen in Zusammenhang mit der Nutzung der DiGA', 'necessity_of_medical_activities'], ['Funktionen der digitalen Gesundheitsanwendung', 'digital_health_application_functions'], ['Angaben zur Evidenz', 'evidence_information'], ['Angaben zum positiven Versorgungseffekt', 'positive_care_effect_information'], ['Beschreibung des medizinischen Nutzens', 'medical_benefit_description'], ['Studie/n im PICO-Schema', 'studies_in_pico_schema'], ['Die DiGA enthält ein diagnostisches Instrument', 'contains_diagnostic_tool'], ['Zusammenfassung zu Studiendesign und Ergebnissen einschließlich Verweises auf den Ort der Registierung der Studie/n', 'study_design_and_results_summary'], ['Dies ist/sind die vom Hersteller angeführten Studie/n:', 'manufacturer_specified_studies'], ['Übertragbarkeit des positiven Versorgungseffekts auf andere Patientengruppe/n', 'transferability_to_other_patient_groups'], ['Institution/en, die am Evaluationskonzept beteiligt waren', 'institutions_involved_in_evaluation_concept'], ['Diese DiGA ist verfügbar in den folgenden Sprachen', 'available_languages'], ['Hinweise für eine sichere Anwendung', 'safe_application_guidelines'], ['Literaturquellen (wie z.B. Leitlinien, Lehrwerke, Studien) für die DiGA', 'literature_sources'], ['Beteiligung von medizinischen Einrichtungen und Organisationen bei der Entwicklung der DiGA', 'medical_institutions_organizations_participation']]
    for i in range(0, len(qa_elements), 2):
        question = qa_elements[i].text
        answer_element = qa_elements[i + 1]
        print("Lolo1: " +str(question))
        print("Lolo: " +str(answer_element))
        answer = answer_element.text
        print(answer, question)
        for x in fachtkreis_ask:
            if x[0] in question:
                answers[x[1]] = answer
                break
        # Get the answer text from the answer element

    bsrf = driver.find_elements("css selector","section.app-section:nth-child(2) > div:nth-child(1) > p")
    true_p = 0
    text_list = ""
    for v in bsrf: 
        if "Basis der vom Hersteller" or "Die Anforderungen an" in v.text :  
            if len(v.text) > (len("Auf Basis der vom Hersteller im Antragsverfahren vorgelegten Angaben und Nachweise bewertet das Bundesinstitut für Arzneimittel und Medizinprodukte (BfArM) die Anforderungen an")+3):
                text_list = v.text
            else: 
                true_p = 1 
                text_list = text_list  + " \n " +  v.text 
        elif true_p:
            if not "Nachfolgend finden Sie Informationen zu den verfügbaren Sprachversionen," in v.text : 
                text_list = text_list  + " \n " +  v.text 
            else: 
                break 
    
    answers["bewertungsentscheidung_des_bfarm"] = text_list

    table = driver.find_element(By.CLASS_NAME, 'entity-prescriptions__table')
    rows = table.find_elements(By.TAG_NAME, 'tr')

    headers = rows[0].find_elements(By.TAG_NAME, 'th')
    header_titles = [header.text for header in headers]

    datatable = []
    for row in rows[1:]:  
        cells = row.find_elements(By.TAG_NAME, 'td')
        datatable  = diga.copy()
        datatable.update({header_titles[k]: cells[k].text for k in range(len(cells))})
        table_data.append(datatable)
    
    # Switch back to the home page
    
    driver.execute_script("window.scrollTo(0, 0);")
    driver.execute_script("window.history.go(-2)")
    


    # Random delay before processing the next element
    time.sleep(random.uniform(1, 2))  # Random delay to mimic human-like behavior
    
    combined_data.update({**info, **answers})
   
    # Add the extracted data to the list
    data.append(combined_data)
   
    
    # Increment the counter
    counter += 1
    
    # Check if the desired number of records is reached
    if counter == 54:
        break
    time.sleep(12)

# Close the WebDriver
driver.quit()

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)
dftable = pd.DataFrame(table_data)


Processing element 1
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
apps.apple.com/de/app/actensio/id1592911608 Anwendungs-URL im Apple App Store Deutschland
Version 1.0.* Versionsnummer iOS-App
entered
iPhone, iPad, iPod Touch mit mindestens:
- 1.8 Ghz CPU (dual core)
- 2 GB RAM
- PowerVR GT7600 GPU (oder ähnlich)
- 8GB Speicher Hardwarekompatibilität iOS-App
iOS 13.5 oder höher Softwarekompatibilität iOS-App
play.google.com/store/apps/details?id=de.mementor.actensio&gl=DE Anwendungs-URL im Google Play Store Deutschland
Version 1.0.* Versionsnummer Android-App
Smartphone/Tablet mit mindestens 1.8 GHz CPU (dual core), 2 GB RAM, PowerVR GT7600 GPU (oder ähnlich), 8GB Speicher Hardwarekompatibilität Android-App
Android 8.0 oder höh

In [9]:
# Aktuelles Datum holen
aktuelles_datum = datetime.now().date().strftime('%Y_%m_%d')

print(aktuelles_datum)
df.to_csv('F_diga_verzeichnis_'+aktuelles_datum+'.csv', encoding='utf-8-sig', index=False, sep=';')

dftable.to_csv('F_diga_verzeichniss_table'+aktuelles_datum+'.csv', encoding='utf-8-sig', index=False, sep=';')

2024_06_16


In [10]:
excel_file_path = 'F_diga_verzeichniss_'+aktuelles_datum+'.xlsx'
#df = pd.read_csv('output.csv', encoding='utf-8')
df.to_excel(excel_file_path, index=False)

excel_file_path = 'F_diga_verzeichniss_table_'+aktuelles_datum+'.xlsx'
#df = pd.read_csv('output.csv', encoding='utf-8')
dftable.to_excel(excel_file_path, index=False)
